# Count CSV Rows in Labels Subfolders

This notebook counts rows (excluding headers) in CSV files within specified labels subfolders (e.g., `labels/jaw` or `labels/tip`).

In [ ]:
from utils import scan_csv_in_labels_subfolders, display_csv_scan_results
import pandas as pd

# Configure pandas to display all rows
pd.set_option('display.max_rows', None)

## Configuration

In [ ]:
# Root directory containing folders with CSV files
ROOT_DIR = r'C:\Users\wanglab\Desktop\Tip_Curve'

# Specify which subfolder within 'labels' to search for CSV files
# Examples: 'jaw', 'tip', 'tongue', etc.
SUBFOLDER_NAME = 'jaw'

## Scan Folders and Display Results

In [ ]:
# Scan the directory structure
scan_data = scan_csv_in_labels_subfolders(ROOT_DIR, SUBFOLDER_NAME)

# Display formatted results
df_with_totals, df_valid = display_csv_scan_results(scan_data)

# Show the full table
display(df_with_totals)

Scanning 57 folders in C:\Users\wanglab\Desktop\Tip_Curve

Found CSV files in 0 cases
Missing or failed: 57
Total rows (excluding headers): 0


## Show Only Valid Results

In [ ]:
# Display only folders where CSV files were successfully found and processed
if len(df_valid) > 0:
    print(f"\nValid CSV files found: {len(df_valid)}")
    display(df_valid)
else:
    print("No valid CSV files found.")